# Equities: enterprise and speculative returns

We first construct a model for equity principal returns
by decomposition into enterprise and speculative returns.
That model is then tested using S&P 500 data going back
to 1871 (well-known database assembled by Robert Shiller).

We find that the percentage approximation model is
not appropriate as the time horizon increases.
In contrast, the exactitude of the logarithmic model
is demonstrated by multivariate regression.

The enterprise and speculative returns are then
plotted to understand their respective relationship
to price dynamics. The Great Recession produced
spikes in the data unlike any other in history.

Total return is therefore the sum of returns based on
enterprise, speculation, and dividends.
We examine the dividend payout ratio relative
to earnings, and find a local estimate for forecasting.

Finally we demonstrate forecasting using the above
decomposition of returns and the most recent data.
Then those results are compared to an univariate
Holt-Winters time-series model in Appendix 1.

Appendices 2 and 3 covers the historical characteristics
of the three separable components of total return.

Reference section also relates our findings to Bogle (2015).

*Dependencies:*

- Repository: https://github.com/rsvp/fecon235
- Python: matplotlib, pandas

*CHANGE LOG*

    2016-03-18  Rough draft, test data source directly.

In [ ]:
from fecon235.fecon235 import *

In [ ]:
#  PREAMBLE-p6.15.1223 :: Settings and system details
from __future__ import absolute_import, print_function
system.specs()
pwd = system.getpwd()   # present working directory as variable.
print(" ::  $pwd:", pwd)
#  If a module is modified, automatically reload it:
%load_ext autoreload
%autoreload 2
#       Use 0 to disable this feature.

#  Notebook DISPLAY options:
#      Represent pandas DataFrames as text; not HTML representation:
import pandas as pd
pd.set_option( 'display.notebook_repr_html', False )
#  Beware, for MATH display, use %%latex, NOT the following:
#                   from IPython.display import Math
#                   from IPython.display import Latex
from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)
#  Or equivalently in config file: "InteractiveShell.display_page = True", 
#  which will display results in secondary notebook pager frame in a cell.

#  Generate PLOTS inside notebook, "inline" generates static png:
%matplotlib inline   
#          "notebook" argument allows interactive zoom and resize.

## Model without interest rates

We will test a model for equity returns which shall
exclude the usual considerations for interest rates,
the discounting of dividends, and the so-called "equity premium."

Let us begin with the obvious:

$$ p_t = x_t s_t \qquad\hbox{where}\quad  s_t = p_t / x_t $$

Price per share at time t will be represented by $p_t$,
and the earnings per share by $x_t$.
Thus $s_t$ is the usual price/earnings ratio.

Taking the log of both sides, we can get two equations for different epochs:

$$ \eqalign{ \log p_t &= \log x_t + \log s_t \cr
   \log p_{t-n} &= \log x_{t-n} + \log s_{t-n} \cr}$$
   
Subtracting the second equation from the first, given $n > 0$, we obtain:

$$ \eqalign{ \log {p_t \over p_{t-n}} &= \log {x_t \over x_{t-n}} + \log {s_t \over s_{t-n}} \cr
   P_t &= X_t + S_t \qquad\hbox{for some implicit  } n.\cr }$$

Note for small $\psi$: $\log(1+\psi) \approx \psi$, where $\psi$
can be expressed in percentage terms.
The capitalized variables thus can be approximately interpreted as follows:
percentage *price change* $P_t$ is equal to percentage *earnings growth* $X_t$
plus the percentage *change in price/earnings ratio* $S_t$ --
over $n$ periods.

As we shall see, $X_t$ can be interpreted as **enterprise return**,
whereas $S_t$ can be characterized as **speculative return**.

## Data specification and retrieval

We shall rely upon Quandl [MULTPL](https://www.quandl.com/data/MULTPL)
database which keeps current the famous series made public by
Robert [Schiller](http://www.econ.yale.edu/~shiller/data.htm)
in his Nobel prize-winning work. Schiller keeps his data
in an [Excel spreadsheet](http://www.econ.yale.edu/~shiller/data/ie_data.xls),
but our downloaded data will be in pandas Dataframe format.
We have also doubled-checked the data with CSV formatted
[okfn.org](http://data.okfn.org/data/core/s-and-p-500)
and its GitHub [datasets](https://github.com/datasets/s-and-p-500)
repository.

Here is **Schiller's description of the data:**
"Stock market data used in my book, *Irrational Exuberance*
[Princeton University Press 2000, Broadway Books 2001, 2nd ed., 2005]
are available for download: U.S. Stock Markets 1871-Present and CAPE Ratio.
This data set consists of **monthly** stock price, dividends, and earnings
data and the consumer price index (to allow conversion to real values),
all **starting January 1871**. The price, dividend, and earnings series
are from the same sources as described in Chapter 26 of my earlier book
(*Market Volatility* [Cambridge, MA: MIT Press, 1989]),
although now I use monthly data, rather than annual data.
Monthly dividend and earnings data are computed from the
S&P four-quarter totals for the quarter since 1926,
with linear interpolation to monthly figures.
Dividend and earnings data before 1926 are from Cowles and associates, 
*Common Stock Indexes* [2nd ed. Principia Press 1939],
interpolated from annual data.
Stock price data are monthly averages of daily closing prices."

For our earnings data, we shall ignore Schiller's CAPE ratio,
and instead derive the raw data from price/earning ratios.

In [ ]:
#  We shall use quandlcode of monthly frequency:
s = getqdl( 'MULTPL/SP500_PE_RATIO_MONTH' )


p  = getqdl( 'MULTPL/SP500_REAL_PRICE_MONTH' )
#                          ^But they meant NOMINAL!
#  Their inflation-adjusted monthly series is called
#  MULTPL/SP500_INFLADJ_MONTH

#  We want non-inflation adjusted data for this notebook.

In [ ]:
#  Get the plain earnings -- which trails over 12-months:
x  = todf( p / s )

In [ ]:
#  Apply numpy log transform, base e:
plog = np.log( p )
xlog = np.log( x )
slog = np.log( s )

### Shape the data: endpoints, n, and transform methods

Begin specific tests from the next cell:

In [ ]:
#  RE-RUN the REGRESSION starting from this cell, varying specs:

#  Raw data starts 1871-01-01, but shall allow lags up to 24 months:

start = '1873-01-01'
end   = '2020-01-01'

#  Compute percentage change over freq of n periods:

n = 12

#  Percentage transform is approximate, but widely used,
#  whereas logarithmic transform gives exact results.
transform = "%"
transform = "log"

In [ ]:
## This is the approximate PERCENTAGE transforms: 

if transform == "%":
    P  = div( pcent(p, n), n ) * 12
    X  = div( pcent(x, n), n ) * 12
    S  = div( pcent(s, n), n ) * 12

#  div() is our numpy function for Python 2 and 3 division.
#  The div() and the 12 annualizes the percentage.

In [ ]:
## This is the exact LOG differencing transforms: 

if transform == "log":
    P  = div( dif(plog, n), n ) * 12
    X  = div( dif(xlog, n), n ) * 12
    S  = div( dif(slog, n), n ) * 12

#  div() is our numpy function for Python 2 and 3 division.
#  The div() and the 12 annualizes the result.

In [ ]:
#  Create a mega Dataframe:
mega = paste([ p, x, s, P, X, S ])

#  Then name the columns:
mega.columns = ["p", "x", "s", "P", "X", "S"]

In [ ]:
stats( mega )

## Apply regression to test  model

The correlation matrix above gives us some partial insight
into pairwise relationships. We would expect P and X
to be strongly correlated.

Next we conduct a multivariate regression by specifying the
form of the theoretical model. We suppress estimation of
the intercept by using "0" within regressformula().
Obviously, we are the coefficients for X and S should
to be close to 1 -- but the results are quite unexpected.

In [ ]:
# Get more info on regression by: regressformula??

result = regressformula( mega[start:end], "P ~ 0 + X + S" )
#                            ^possible truncation of Dataframe

In [ ]:
print( result.summary() )

In [ ]:
print("Confirming: transform=", transform, ", n=", n, ", start=", start, ", end=", end)

## Warning: percentage approximation is terrrible

In many studies, the percentage approximation is used to examine returns,
perhaps because conceptually it is readily understood.
By executing the above regression while *commenting out* the **transform="log"** line,
we can replicate the following results for various n values. As n increases,
the R-squared statistic worsens considerably and the coefficients deviate farther away
from the theoretical value of 1. Let $\varepsilon_t$ be the estimation error, then:

$$\eqalign{
\varepsilon_t &= \hat{P_t} - P_t \cr
\hat{P_t} &= 0.78 X_t + 0.91 S_t + \varepsilon_t \qquad \hbox{for  } n=1 \quad R^2=0.90 \cr
\hat{P_t} &= 0.41 X_t + 0.61 S_t + \varepsilon_t \qquad \hbox{for  } n=3 \quad R^2=0.57 \cr
\hat{P_t} &= 0.19 X_t + 0.30 S_t + \varepsilon_t \qquad \hbox{for  } n=6 \quad R^2=0.27 \cr
\hat{P_t} &= 0.15 X_t + 0.23 S_t + \varepsilon_t \qquad \hbox{for  } n=12 \quad R^2=0.21 \cr
}$$

The errors can be traced to the data reality that $\psi$
cannot be assumed to be small, for there were indeed some
extreme changes in the course of 143 years.
Lengthening $n$ also contributes to violating that
numerical approximation.

## Exact results using log transform

Thankfully the data (and algorithms) confirm the mathematical
derivation of our model when the **transform="log"** line is in effect:

$$\eqalign{
\hat{P_t} &= 1.00 X_t + 1.00 S_t + \varepsilon_t \qquad \hbox{for all  } n \quad R^2=1.00 \cr
}$$

Increasing $n$ has no effect on our results, which was entirely expected.
This can be verified by re-executing the appropriate sections
of this notebook with different parameters.

In view of our findings, we make the this revision:
**the *logarithmic* price change $P_t$ is *exactly* equal to the
*logarithmic* earnings change $X_t$
plus the change in *logarithmic* price/earnings ratio $S_t$ --
over $n$ periods.**

The pairwise correlations, however, were somewhat surprisingly low.
For example, for $n=12$, the correlation between $P_t$ and $X_t$
was 0.37, and the correlation between $P_t$ and $S_t$ was 0.28.
But of course, as an ensemble, they mesh together perfectly.

## Further regarding X and S

The two variables $X_t$ and $S_t$ have the same functional form
and the same coefficient weights, yet
the mean of the former is 0.03781 and
the mean of the latter 0.00432,
which differs by a factor of 8.75 (when $n=12$).
This indicates variation in earnings contributes far more
to price dynamics than variation in the price/earnings ratio.

Since our data has monthly frequency, we shall focus on
cases where $n$ is a multiple of 12.
This is to mitigate the possibility of observing seasonal effects.

Since the logarithms of the raw data are clearly crucial,
let us see their stats:

In [ ]:
rawlog = paste([plog, xlog, slog])
rawlog.columns = ["pl", "xl", "sl"]
rawlog = rawlog[start:end]
#  We intentionally use "start" and "end"
#  to remain consistent with above analysis.

stats( rawlog )

Next we visualise their history:

In [ ]:
plot( rawlog.pl )

In [ ]:
plot( rawlog.xl )

In [ ]:
plot( rawlog.sl )

Looking for possible outliers, we are struck by the violent downward spike
occuring during the Great Recession in the earnings data.

Looking for possible trends, we can see how the price dynamics
is fundamentally related to earnings growth. In fact,
the **correlation between pl and xl is 0.98** -- which
justifies calling *earnings growth* $X_t$: the **enterprise return**
portion of total return (to be discussed in the final sections).

Prior to 1989, sl looked almost trendless, bouncing roughly
between 2 and 3. Now raw s is the price/earnings ratio,
which is basically the multiplier to earnings at which
an "investor" has implicitly paid at market.
That investor's optimism for further growth perhaps can
be encapsulated by the change in valuation as
change in the price/earnings rato.
This is the reason behind calling $S_t$
the **speculative return**.

In [ ]:
#  This is raw price/earnings:
stat( s )

The max on raw price/earnings is 123.73 which is not an
outlier but due to the dramatic drop in earnings
during the Great Recession (quite evident from the plots).
By moderating the time window to before the year 2000, we estimate the
all-time mean price/earnings to be around 15
(note $log 15 = 2.71$ and our remark on trendless sl),
and see the standard deviation drop from
8.4 to 4.6.

In [ ]:
stat( s[:'2000'] )

## Dividends

Roughly speaking, dividends are taken out of earnings and paid out to investors.
We now recover the raw data.

In [ ]:
#  We shall use quandlcode of monthly frequency:
dyield = getqdl( "MULTPL/SP500_DIV_YIELD_MONTH" )

In [ ]:
#  Unfortunately, dyield uses last day of month convention,
#  instead of first day of month like s and p, therefore:
dyield = dyield.set_index(pd.DatetimeIndex([i.replace(day=1) for i in dyield.index]))

In [ ]:
#  dyield is in percentage terms, thus we will
#  need to convert to obtain raw dividend data.
d = todf((dyield / 100.00) * p)

In [ ]:
## Uncomment to see raw 12-month dividend:
#  d

In [ ]:
#  We want to compute the PAYOUT ratio = dividends/earnings
divearn = todf( d / x )

In [ ]:
## Uncomment to see payout ratio data:
#  divearn

In [ ]:
plot( divearn )

In [ ]:
stat( divearn )

On average, we would expect divearn to be less than 1, since
earnings are the main source for paying out the dividends.

Sometimes, however, firms may hold steady their dividends
while earnings plunge -- for example, during the
Great Recession, the payout ratio went as high as 4.
But as the plot shows this is highly unusual.

Historically we can observe a slight downward trend in the payout ratio
over the very long-term. The mean payout ratio is 0.61.

# TOTAL RETURN for equities

The total return $T_t$ for equities is composed of the logarithmic price change
plus the dividend yield, over $n$ periods. We shall let $\lambda$ be
the dividend payout ratio relative to earnings $x_i$,
such that the index $i$ is understood to cover periods $t-n$ through $t$.
Thus, we can write:

$$ T_t = P_t + {{\lambda \sum x_i } \over p_{t-n}} $$

Inserting our model in place of $P_t$, we obtain total return as follows:

$$ T_t = X_t + S_t + {{\lambda \sum x_i } \over p_{t-n}} $$

# Forecasting total return

The term lambda $\lambda$ should be a stochastic variable, but we shall
simplify it to a some constant as an approximation.
We utilize the median, rather than the mean, since
that is a far more robust statistic.

To estimate the other variables we localize our data to
the last three years, which gives 36 monthly points of data.

In [ ]:
#  For localizing forecasts:
nlocal = 36

### Estimating future dividend yield

In [ ]:
#  Estimated local lambda:
Elambda = np.median( tail(divearn, nlocal) )
Elambda

In [ ]:
#  Estimated dividend yield:
Edyield = Elambda * (tailvalue(x) / tailvalue(p))
Edyield

### Estimating earning growth

We tested our model using the change in logarithmic earnings,
but here we take a shortcut by using the geometric mean return
on the localized raw data.

In [ ]:
#  Get more info by executing: georet??
Xgeoret = georet( tail(x, nlocal), yearly=12 )
Xgeoret

In [ ]:
#  Geometric return was expressed in percentage terms,
#  so here our estimator for big X:
EX = Xgeoret[0] / 100.00
EX

### Estimating growth of price/earnings ratio

We tested our model using the change in logarithmic price/earnings ratio,
but here we take a shortcut by using the geometric mean return
on the localized raw data.

In [ ]:
#  yearly argument in georet annualizes:
Sgeoret = georet( tail(s, nlocal), yearly=12 )
Sgeoret

In [ ]:
#  Geometric return was expressed in percentage terms,
#  so here our estimator for big S:
ES = Sgeoret[0] / 100.00
ES

## Combining the estimates for a forecast

In terms of our Python variables, here is our model:

$$ \hat{T} = \hat{X} + \hat{S} + \hat{dy} $$

where an "E" prefix is used to represent estimated hat variables.

In [ ]:
# FORECAST of TOTAL RETURN, finally...
ET = EX + ES + Edyield
ET

### 2016-03-18 Commentary

- **Total return for equities is forecasted at 8.76% per annum.**
- However, speculative return ES is 67% responsible for that total return.
- Enterprise return is weak: earnings growth is below the inflation rate!
- Current price/earnings stands at 21.0 (compared to 15 historically).
- Forecast is conditional on price/earnings increasing to 22.2 in a year.
- Estimated dividend yield is 1.74% per annum.

In [ ]:
#  Current price/earnings ratio:
tailvalue( s )

## Appendix 1: Comparison to univariate forecast

Let us apply the Holt-Winters method to logarithmic price alone.
This is a pure time-series technique for the purpose of comparison.

In [ ]:
#  Forecast 12-months ahead:
forecast( plog, h=12 )

In [ ]:
#  Forecast 12-months ahead:
fc = forecast( plog, h=12 )

In [ ]:
#  Change in logarithmic price:
fc.loc[12] - fc.loc[0]

2016-03-18: The forecasted price return is about -5.31% per annum,
so adding the estimated divident yield of +1.74%,
the forecasted total return is -3.57%.
Thus the univariate method differs considerably from
our model which forecasted +8.76% for total return.

*Averaging forecasts from different techniques usually increases accuracy.*
Specifically here: the combined forecasts for total return
is +2.6%.


## Appendix 2: Long-term enterprise and speculative returns

In [ ]:
#  Enterprise return since 1871:
georet( x, yearly=12 )

In [ ]:
#  Speculative return since 1871:
georet( s, yearly=12 )

By examining their geometric mean returns,
we see that enterprise returns (3.62%) contributes far more
to the total return than speculative returns (0.42%).

Our georet() also computes arithmetic mean return
and its standard deviation (second and third
elements respectively). Speculative returns
contributes more volatility in observed
total returns than enterprise returns.

## Appendix 3: Dividend yield historically

Among the three components of total return,
the dividend yield has on average has made
the largest contribution historically.
Also, it has never contributed negatively
to total return.

In [ ]:
#  Dividend yield here is expressed as percentage:
stat( dyield )

The average dividend yield since 1871 has been 4.4% per annum.
But the plot of the exponentially smoothed values seems to
indicate a steady decline over time.

Continuance of such a decline in dividend yield would then make
earnings growth the main driver of total return
in the 21st century.

In [ ]:
plot( ema(dyield, 0.10) )

## References

- Bogle, John C. and Michael W. Nolan, Jr., 2015, "Occam’s Razor Redux: Establishing Reasonable Expectations for Financial Market Returns," *Journal of Portfolio Management* 42, 119-134. http://www.iijournals.com/doi/full/10.3905/jpm.2015.42.1.119 


- Keynes, John Maynard, 1936, *General Theory of Employment, Interest, and Money*, MacMillan, London.

Keynes (1936) enumerated two sources of stock returns: **enterprise** and **speculative**.
Enterprise refers to the actual business results of corporations.
Speculation, on the other hand, involved “forecasting the psychology of the market,”
the ephemeral, emotional response of investors seeking to
anticipate how other investors will react to particular events.

Keynes made no known attempt to quantify the distinction between
enterprise and speculation, so Bogle tackled the task
by developing his BSRM/S model,
"*Bogle Sources of Return Model for Stocks*." 

Bogle (2015) states: *"**The sheer simplicity of the BSRM/S commends it over
highly complex models, none of which (as far as we can determine)
has offered similar accuracy.**"* This is quite a claim since Bogle
has been involved in finance since 1951 (Princeton honors thesis),
and is the founder of The Vanguard Group.

Bogle (2015) does not show how he arrived at his model mathematically:
"Although we find this validation of the model gratifying,
we suspect a fit this close to the actual total return has a lot to do with *luck*."
Perhaps he was lucky in terms of realized out-of-samples results,
but the analytics as shown in this notebook is sound.

#### Summary

John Bogle sought a deeper reasoning behind his expectations
of future total return from a portfolio of equities.
A sample explanation could be phrased as follows:
"I expect annual earning growth of 8%,
and in ten years a terminal price/earning ratio of 14 --
given that ratio is 15.5 currently,
such a 1% reduction per annum will decrease
the total return by like amount.
Therefore, with a dividend yield of 3%,
the my expected total return is 8% - 1% + 3% = 10%
over the coming decade."

This notebook shows the percentage approximate reasoning
is prone is large errors, whereas a logarithmic version
is exact. The data since 1871 allows us to examine
the separable components of total return:
enterprise and speculative returns, plus dividend yield.
